<h1 style="font-size:42px; text-align:center; margin-bottom:30px;"><span style="color:SteelBlue">TM1py:</span> Outliers detection</h1>
<hr>

The first step you should complete before actually stepping through this code is to install all required dependencies. Assuming you already have Anaconda installed, you'll  need to install the following libraries:
<pre>pip install TM1py</pre>
<pre>pip install scikit-learn</pre>
<pre>pip install matplotlib</pre>

## Step 1: Import the required modules

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

## Step 2: Setup your TM1 parameters

### TM1 Server Parameters
Populate your TM1 server parameters below :

In [2]:
#Server address
address = 'localhost'
#HTTP port number - this can be found in your config file
port = '8892'
#username
user = 'admin'
#password
password = 'apple'
#SSL parameter - this can be found in your config file
ssl = True

## Step 3: Get TM1 Data
The following code obtains a data set from TM1 based on the cube and view specified above. The resulting dataset is transformed into a pandas dataframe.

Before importing any data, create the view you would like to import into the data frame. Populate the fields below.

In [3]:
#specify the cube
cube_name = 'Sales'
#specify the view
view_name = 'Sales_2021_2022'

In [4]:
tm1 = null
with TM1Service(address= address, port=port, user=user, password=password, ssl=ssl) as tm1:
    
    # Extract pnl data from specified cube view
    raw_data = tm1.cubes.cells.get_view_content(cube_name=cube_name, view_name=view_name, private=False)

    # Build pandas DataFrame fram raw cellset data
    df = Utils.build_pandas_dataframe_from_cellset(raw_data, multiindex=False)

## Step 4: Outlier detection

First, we select the columns PRICE, QUANTITY and SKU as features for clustering.
Afterwards, the data is normalized using StandardScaler to ensure that each feature contributes equally to the clustering process.
DBSCAN is applied to the normalized data. The parameters eps=0.5 and min_samples=5 control the density required to form a cluster. You can tune these values based on your data distribution.
The "Cluster" column is added to the original dataset to store the cluster labels for each data point. Points labeled -1 are considered outliers.

In [5]:
# Select relevant columns for clustering (PRICE, QUANTITY, SKU)
features = df[['PRICE', 'QUANTITY', 'SKU']]


# Normalize the features using StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Apply DBSCAN algorithm
# You can adjust eps (radius of neighborhood) and min_samples (min points to form a cluster)
dbscan = DBSCAN(eps=0.5, min_samples=5)  # You can tune eps and min_samples based on your data
clusters = dbscan.fit_predict(features_scaled)

# Add the cluster labels to the original dataframe
df['Cluster'] = clusters

A scatter plot shows the clustering result based on PRICE and QUANTITY, where different clusters are color-coded. The color bar indicates the cluster labels, with -1 representing outliers.

In [6]:
# Visualize the clusters using PRICE and QUANTITY
plt.figure(figsize=(10, 6))
plt.scatter(df['PRICE'], df['QUANTITY'], c=df['Cluster'], cmap='plasma')
plt.title('DBSCAN Clustering of Sales Data')
plt.xlabel('Price')
plt.ylabel('Quantity')
plt.colorbar(label='Cluster')
plt.show()

# Filter and display only the outliers (where Cluster = -1)
outliers = df[df['Cluster'] == -1]

<img src="https://raw.githubusercontent.com/missaoui/TM1-outlier-detection/main/Outlier2310.png" />


The following table shows all the outliers : 

<img src="https://raw.githubusercontent.com/missaoui/TM1-outlier-detection/main/Outliers table.jpg" />


Let's analyse why we have 3 lines as outliers for SKU 3. The boxplot shows the distribution of prices for SKU 3. The median price is around 35-40, with a minimum close to 17 and a maximum reaching 70. The box (IQR) shows that the majority of prices are clustered between 30 and 50, with some variance but no extreme outliers in price.

<img src="https://raw.githubusercontent.com/missaoui/TM1-outlier-detection/main/boxplotprice_SKU3.png" />

The boxplot for quantity shows more variance, with a median around 200. There are many outliers, with quantities reaching up to 810, but most transactions for SKU 3 occur between 100 and 300. This suggests that while some large purchases occur, they are not frequent. This is why the DBSCAN algorithm detects three lines as outliers.

<img src="https://raw.githubusercontent.com/missaoui/TM1-outlier-detection/main/boxplotquanity_SKU3.png" />

## Step 5: Send the outliers into TM1

In [ ]:
# Prepare outliers for insertion into the cube
# Example cube dimensions: Version, Period, SKU, Customer, Geography, Measures
version = "Actual"
measure = "Outlier"

# Iterate through the outliers and build the data for TM1 insertion
data_to_insert = []
for _, row in outliers.iterrows():
    period = row['DATE'].strftime('%Y-%m')  # Assuming period is in YYYY-MM format
    sku = str(row['SKU'])
    customer = str(row['CUSTOMER'])
    geography = str(row['CITY'])
    
    # The outlier flag (e.g., 1 for outlier) will be stored in the Measures dimension as 'Outlier'
    data_to_insert.append({
        ("Actual", period, sku, customer, geography, "Outlier"): 1  # 1 indicates an outlier
    })

# Define the cube name (replace 'Sales_Cube' with your actual cube name)
cube_name = 'Sales_Cube'

# Insert the outlier data into the TM1 cube
for data in data_to_insert:
    tm1.cubes.cells.write_values(cube_name, data)

print("Outliers successfully inserted into TM1 cube!")

# Disconnect from TM1
tm1.logout()
